<a href="https://colab.research.google.com/github/nneko1231/RoopColab/blob/main/DeepFake_V2.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Drive Mount

In [ ]:
# will mount drive if MOUNT_DRIVE is True
import os
import subprocess
import shutil
from google.colab import drive, output

# Add a parameter to control mounting Google Drive
MOUNT_DRIVE = True  #@param {type:"boolean"}

# mount drive if MOUNT_DRIVE is True
if MOUNT_DRIVE:
    drive.mount('/content/drive')

# 2. Installing

In [ ]:
!git clone https://github.com/nneko1231/roop
%cd roop
!pip install onnxruntime-gpu && pip install -r requirements.txt

In [ ]:
!wget https://huggingface.co/ezioruan/inswapper_128.onnx/resolve/main/inswapper_128.onnx -O inswapper_128.onnx
!mkdir models
!mv inswapper_128.onnx ./models

# 3. Settings

In [ ]:
SOURCE_PATH = "Input.mp4" #@param {type:"string"}
TARGET_PATH = "Target.jpg" #@param {type:"string"}
OUTPUT_PATH = "Output.mp4" #@param {type:"string"}
keep-fps = True  #@param {type:"boolean"}
keep-frames = True  #@param {type:"boolean"}
skip-audio = False  #@param {type:"boolean"}
temp-frame-format = "jpg" #@param ["jpg" , "png"]
temp-frame-quality = 100 # @param {"type":"slider","min":1,"max":100,"step":1}
output-video-encoder = "h264_nvenc" #@param ["libx264","libx265","libvpx-vp9","h264_nvenc","hevc_nvenc"]
output-video-quality = 100 # @param {"type":"slider","min":1,"max":100,"step":1}
max-memory MAX_MEMORY = 12 # @param {"type":"slider","min":0.1,"max":12,"step":0.1}


# 4. Run

In [ ]:
def build_backend_command() -> str:
    command = (('python3 real-video-enhancer/backend/rve-backend.py')
        f'-s "{SOURCE_PATH}" '
        f'-t "{TARGET_PATH}" '
        f'-o "{OUTPUT_PATH}" '
        f'--execution-provider cuda '  # Always use CUDA for GPU acceleration
        f'--frame-processor face_swapper face_enhancer '  # Always use face_swapper and face_enhancer
        + f'--keep-fps {str(keep_fps).lower()} '  # Keep FPS flag
        + f'--keep-frames {str(keep_frames).lower()} '  # Keep frames flag
        + f'--skip-audio {str(skip_audio).lower()} '  # Skip audio flag
        + f'--temp-frame-format {temp_frame_format} '  # Temp frame format (jpg or png)
        + f'--temp-frame-quality {temp_frame_quality} '  # Temp frame quality (0-100)
        + f'--output-video-encoder {output_video_encoder} '  # Output video encoder (e.g., h264_nvenc)
        + f'--output-video-quality {output_video_quality} '  # Output video quality (0-100)
        + f'--max-memory {max_memory} '  # Max memory usage in GB
    )
    return command

# Write the command to a shell script
with open("command.sh", "w") as f:
    f.write(build_backend_command())

# Run the command from the shell script
!bash command.sh

# Remove the command.sh file after execution
import os
os.remove("command.sh")